In [1]:
import json

import numpy as np
import torch
import torch.nn as nn
from einops import rearrange
from torch.utils.data import DataLoader
from yacm import ConfigMixin, register_to_config

from meshae.config import MeshAEModelConfig
from meshae.dataset import MeshAEDataset, MeshAECollateFn
from meshae.model import MeshAEModel
from meshae.utils import quantize, tensor_describe

In [2]:
class MyModel(nn.Module, ConfigMixin):
    config_name = "my_model_config.json"
    ignore_for_config = []

    @register_to_config
    def __init__(self, hidden_size: int = 768, _num_layers: int = 12, dropout: float = 0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = _num_layers
        self.dropout = dropout  # This will be ignored because of the specification in `ignore_for_config`

model = MyModel(hidden_size=1024, _num_layers=20)
model.config

FrozenDict([('_use_default_values', ['dropout']),
            ('hidden_size', 1024),
            ('dropout', 0.1)])

In [ ]:
print(model.save_config())

{
  "_class_name": "MyModel",
  "_use_default_values": [
    "dropout"
  ],
  "dropout": 0.1,
  "hidden_size": 1024
}


In [5]:
with open("../configs/default/config.json") as fin:
    config = MeshAEModelConfig.from_dict(json.load(fin))

state_dict = torch.load("../checkpoints/default/ckpt-01-0100.pt").get("model_state_dict")
model = MeshAEModel(**config.to_dict())
model.load_state_dict(state_dict)
# model = model.to("cuda")

collate_fn = MeshAECollateFn()
dataset = MeshAEDataset("../data/objaverse/train", sort_by="zyx")
dataset.objects = dataset.objects[350 * 8:550*8]
dataloader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True,
    num_workers=1,
)

FileNotFoundError: [Errno 2] No such file or directory: '../checkpoints/default/ckpt-01-0100.pt'

In [ ]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        # batch = {k: v.to("cuda") for k, v in batch.items()}
        loss, (recon_loss, commit_loss), logits, coords = model(**batch)
        print(i, loss, recon_loss, commit_loss)
        break

0 tensor(3.9983) tensor(3.9872) tensor(0.0111)


In [13]:
tensor_describe(model.decoder.proj_refine[2].weight)

{'avg': -3.4784083254635334e-05,
 'std': 0.03655720874667168,
 'min': -0.1773606538772583,
 'max': 0.19272103905677795,
 'p25': -0.02468550205230713,
 'p50': -2.3601360226166435e-05,
 'p75': 0.024616852402687073,
 'norm': 56.149593353271484,
 'numel': 2359296,
 'numel_nonzero': 2359296,
 'numel_nonzero_pct': 100.0}

In [5]:
qogits = rearrange(logits, "b ... q -> b q (...)").log_softmax(1)
qoords = rearrange(
    quantize(coords, high_low=config.feature_configs["vrtx"].high_low, num_bins=64),
    "b ... -> b 1 (...)",
)
qoords = torch.zeros_like(qogits).scatter(1, qoords, 1.0)